In [1]:
import pandas as pd
import numpy as np
import os
import duckdb 

In [2]:
""" 
1. GUO cinesi e tutte le loro subsidiaries con codice NACE a 4 digits, paese e città 
(e, se c'è una divisione sub nazionale ma è importante solo per le subs cinesi). 
Oltre a questo ci serve GUO-name e SUBS- name e infine codice Nace del GUO.
2. Subsidiaries cinesi con info su città, NACE a 4 digits e paese del GUO."""

" \n1. GUO cinesi e tutte le loro subsidiaries con codice NACE a 4 digits, paese e città \n(e, se c'è una divisione sub nazionale ma è importante solo per le subs cinesi). \nOltre a questo ci serve GUO-name e SUBS- name e infine codice Nace del GUO.\n2. Subsidiaries cinesi con info su città, NACE a 4 digits e paese del GUO."

In [ ]:
historicals_path = "..\\..\\data_raw\\ownership_history\\links_2016\\part-00000-0a8d598b-34d5-46fe-8e4e-12a02206fbc1-c000.snappy.parquet" 
key_financials_path = "..\\..\\data_raw\\key_financials\\key_financials_eur\\part-00001-d5e63738-ec0f-45f1-9e6e-e90c27a22308-c000.snappy.parquet"
detailed_financials_path = "..\\..\\data_raw\\deatailed_financials\orbis_monthly_industry_global_financials_and_ratios_eur\\part-00000-888b92de-afa9-4abb-83c2-a254fb0aaf28-c000.snappy.parquet"
legal_info_path = "..\\..\\data_raw\\firmographics\\legal_info\\part-00000-7ace73a0-3def-48c9-9b7c-99284c0e0049-c000.snappy.parquet"
listed_financials_path = "..\\..\\data_raw\listed_financials\detailed_format_industries_eur\part-00000-a7a75a95-7a70-4839-be6b-44779c99453c-c000.snappy.parquet"
contact_info = "..\\..\\data_raw\\firmographics\\contact_info\\part-00074-54d921cc-a1c4-4ba5-941f-d71e06e76b2d-c000.snappy.parquet" 
all_addresses_path = "Z:\\dati_moody\\data_raw\\firmographics\\all_addresses\\part-00000-71b72697-2833-4854-947e-f1f88167681a-c000.snappy.parquet"
industry_classification_path = "Z:\\dati_moody\data_raw\\firmographics\industry_classifications\\part-00000-5c9d4193-0803-46e3-aeeb-e01cda8ffc9b-c000.snappy.parquet"  
legal_info_processed = "Z:\\dati_moody\\data_processed\\firmographics_processed\\legal_info\\AG.parquet"
all_addresses_processed = "Z:\\dati_moody\\data_processed\\firmographics_processed\\all_addresses\\AG.parquet"

<>:3: SyntaxWarning: invalid escape sequence '\o'
<>:5: SyntaxWarning: invalid escape sequence '\l'
<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\o'
<>:5: SyntaxWarning: invalid escape sequence '\l'
<>:9: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8900\343522340.py:3: SyntaxWarning: invalid escape sequence '\o'
  detailed_financials_path = "..\\..\\data_raw\\deatailed_financials\orbis_monthly_industry_global_financials_and_ratios_eur\\part-00000-888b92de-afa9-4abb-83c2-a254fb0aaf28-c000.snappy.parquet"
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8900\343522340.py:5: SyntaxWarning: invalid escape sequence '\l'
  listed_financials_path = "..\\..\\data_raw\listed_financials\detailed_format_industries_eur\part-00000-a7a75a95-7a70-4839-be6b-44779c99453c-c000.snappy.parquet"
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8900\343522340.py:9: SyntaxWarning: invalid escape seque

In [4]:
df = pd.read_parquet(historicals_path)

In [5]:
for i in df.columns:
    print(i)

subsidiary_bvd_id
subsidiary_independence_indicator
shareholder_bvd_id
shareholder_independence_indicator
direct_
direct_only_figures_
total_
total_only_figures_
information_date
source
type_of_relation
active_archived
guo_25
guo_50
guo_50c
guo_25c
guo_25_jo
guo_25c_jo


In [6]:
df

,subsidiary_bvd_id,subsidiary_independence_indicator,shareholder_bvd_id,shareholder_independence_indicator,direct_,direct_only_figures_,total_,total_only_figures_,information_date,source,type_of_relation,active_archived,guo_25,guo_50,guo_50c,guo_25c,guo_25_jo,guo_25c_jo
0,RO15110869,B+,RO*110021000020,-,50.00,50.00,n.a.,None,20160809,VD,GUO 25,active,RO*110021000020,None,None,RO15110869,None,None
1,RO15110869,B+,RO*110021000020,-,50.00,50.00,n.a.,None,20160809,VD,DUO 25,active,RO*110021000020,None,None,RO15110869,None,None
2,RO15110869,B+,RO15110869,B+,n.a.,None,n.a.,None,20161231,VD,GUO 25C,active,RO*110021000020,None,None,RO15110869,None,None
3,RO15110869,B+,RO15110869,B+,n.a.,None,n.a.,None,20161231,VD,DUO 25C,active,RO*110021000020,None,None,RO15110869,None,None
4,RO15110869,B+,RO*110021000020,-,50.00,50.00,n.a.,None,20160809,CC,ISH,active,RO*110021000020,None,None,RO15110869,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148795,RO24439373,B+,RO*554033791,-,50.00,50.00,n.a.,None,20160809,CC,SHH,active,RO*554033790,None,None,RO24439373,RO*554033791,None
3148796,RO24439373,B+,RO*554033790,-,50.00,50.00,n.a.,None,20160809,VD,GUO 25,active,RO*554033790,None,None,RO24439373,RO*554033791,None
3148797,RO24439373,B+,RO*554033791,-,50.00,50.00,n.a.,None,20160809,VD,GUO 25,active,RO*554033790,None,None,RO24439373,RO*554033791,None
3148798,RO24439373,B+,RO24439373,B+,n.a.,None,n.a.,None,20161231,VD,GUO 25C,active,RO*554033790,None,None,RO24439373,RO*554033791,None


In [23]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\..\\data_processed\\key_financials_detailed_processed"

def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")

    COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"

    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            {year} AS year,

            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            
            key_financials_detailed.number_of_employees
        FROM 
            '{path}' AS main

        LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS}' AS key_financials_detailed
        ON 
            main.guo_25 = key_financials_detailed.bvd_id_number
        AND 
            key_financials_detailed.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_guo
        ON 
            main.guo_25 = firmographics_guo.bvd_id_number

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."guo_25" LIKE '{country}%'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df


In [ ]:
country = "CH"
year = "2012"
path = "..\\..\\data_raw\\ownership_history\\links_2012\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)

In [8]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\data_processed\\key_financials_detailed_processed"
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\key_financials_processed"

path = "..\\data_raw\\ownership_history\\links_2017\\*.parquet"
year = 2012 
country = "CH"

COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
COUNTRY_KEY_FINANCIALS_DETAILED = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"
COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"\\key_financials_{country}.parquet"


query_detailed = f"""
    SELECT
        main.subsidiary_bvd_id,
        main.guo_25,
        {year} AS year,

        firmographics_sub.nuts2 AS subsidiary_nuts2,
        firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
         
        key_financials_detailed.number_of_employees
    FROM 
        '{path}' AS main

    LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS_DETAILED}' AS key_financials_detailed
        ON 
            main.guo_25 = key_financials_detailed.bvd_id_number
        AND 
            key_financials_detailed.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

    WHERE 
        main."type_of_relation" = 'GUO 25'
    AND 
        main."subsidiary_bvd_id" LIKE '{country}%'
"""

# conn = duckdb.connect()
# df = conn.execute(query_detailed).fetchdf()


In [ ]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\data_processed\\key_financials_detailed_processed"
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\key_financials_processed"

path = "..\\data_raw\\ownership_history\\links_2017\\*.parquet"
year = 2012
country = "CH"

COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
COUNTRY_KEY_FINANCIALS_DETAILED = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"
COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"\\key_financials_{country}.parquet"


query_detailed = f"""
    SELECT
        main.subsidiary_bvd_id,
        main.guo_25,
        {year} AS year,

        firmographics_sub.nuts2 AS subsidiary_nuts2,
        firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
        
        key_financials_detailed.number_of_employees,
        CASE 
            WHEN key_financials_detailed.bvd_id_number IS NOT NULL THEN 'key_financials_detailed'
            ELSE 'none'
        END AS source_key_financials,
        CASE 
            WHEN firmographics_sub.bvd_id_number IS NOT NULL THEN 'firmographics_sub'
            ELSE 'none'
        END AS source_firmographics
    FROM 
        '{path}' AS main

    LEFT JOIN 
        '{COUNTRY_KEY_FINANCIALS_DETAILED}' AS key_financials_detailed
        ON main.guo_25 = key_financials_detailed.bvd_id_number
        AND key_financials_detailed.year = {year}

    LEFT JOIN 
        '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

    WHERE 
        main."type_of_relation" = 'GUO 25'
    AND 
        main."subsidiary_bvd_id" LIKE '{country}%'
"""

conn = duckdb.connect()
df = conn.execute(query_detailed).fetchdf()

## FIRMOGRAPHICS

In [5]:
def get_firmographics_data():
    query = f"""
        SELECT * 
        FROM '{TEMP_TABLE_FIRMOGRAPHICS}'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df



In [ ]:
df = pd.read_parquet(COUNTRY_FIRMOGRAPHICS)

In [43]:
italy_firmographics = pd.read_parquet(COUNTRY_FIRMOGRAPHICS)
italy_firmographics.to_csv("italy_firmographics.csv", index=False)

In [44]:
italy_key_financials_detailed = pd.read_parquet(COUNTRY_KEY_FINANCIALS_DETAILED)
italy_key_financials_detailed.to_csv("italy_key_financials_detailed.csv", index=False)

In [35]:
query_non_detailed = f"""
    SELECT
        main.subsidiary_bvd_id,
        main.guo_25,
        {year} AS year,

        firmographics_sub.nuts2 AS subsidiary_nuts2,
        firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
         
        key_financials_detailed.number_of_employees
    FROM 
        '{path}' AS main

    LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS}' AS key_financials
        ON 
            main.guo_25 = key_financials.bvd_id_number
        AND 
            key_financials.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

    WHERE 
        main."type_of_relation" = 'GUO 25'
    AND 
        main."subsidiary_bvd_id" LIKE '{country}%'
"""

conn = duckdb.connect()
df2 = conn.execute(query_detailed).fetchdf()

In [ ]:
# sort by bvd
df = df.sort_values(by = "subsidiary_bvd_id")
df

In [ ]:
# count diplicated in subsidiary_bvd_id
df.subsidiary_bvd_id.duplicated().sum()


In [ ]:
path1 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_1.csv"
path2 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_2.csv"
path3 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_3.csv"
path4 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_4.csv"
path5 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_5.csv"

de1 = pd.read_csv(path1)
de2 = pd.read_csv(path2)
de3 = pd.read_csv(path3)
de4 = pd.read_csv(path4)
de5 = pd.read_csv(path5)

de = pd.concat([de1, de2, de3, de4, de5])
de

In [ ]:
# check duplicates
print(de.shape)

de_ = de.drop_duplicates()

print(de_.shape)

In [ ]:
path1 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_1.csv"
path2 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_2.csv"
path3 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_3.csv"
path4 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_4.csv"
path5 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_5.csv"

it1 = pd.read_csv(path1)
it2 = pd.read_csv(path2)
it3 = pd.read_csv(path3)
it4 = pd.read_csv(path4)
it5 = pd.read_csv(path5)

it = pd.concat([it1, it2, it3, it4, it5])
it

In [ ]:
import duckdb

query = f"""
SELECT *
FROM parquet_scan('{key_financials_detail_path}')
LIMIT 10
"""

conn = duckdb.connect()
df = conn.execute(query).fetchdf()

print(df)


In [ ]:
df = pd.read_parquet(key_financials_detail_path)
df.columns

In [ ]:
df.columns

In [ ]:
for col in df.columns:
    print(col)


In [ ]:
cols = ['bvd_id_number', 'operating_revenue_turnover_', 'number_of_employees', 'costs_of_goods_sold', 'closing_date']
df[cols].head()

In [ ]:
df.loc[(df.type_of_relation=='ISH') | (df.type_of_relation=='GUO 25')]

In [ ]:
subdirs = [d for d in os.listdir('ownership_history') if 'links' in d and os.path.isdir(os.path.join('ownership_history', d))]

files_dict = {}

for subdir in subdirs:
    subdir_path = os.path.join('ownership_history', subdir)
    files_dict[subdir] = [os.path.join(subdir_path, f) for f in os.listdir(subdir_path)]

In [ ]:
path = files_dict['links_2007'][0]

In [ ]:
test = pd.read_parquet(path)

In [ ]:
test.to_csv('test.csv', index=False)

In [ ]:
test

In [ ]:
import dask.dataframe as dd

european_countries = ['GB', 'FR', 'DE', 'IT', 'ES', 'NL', 'BE', 'PL', 'SE', 'DK', 'FI', 'PT', 'GR', 'IE', 'AT', 'CH', 'NO']

# Extract the first two characters of 'bvd_id_number' and filter based on European countries


test['subsidiary_country_code'] = test['subsidiary_bvd_id'].str[:2]
test['guo_country_code'] = test['shareholder_bvd_id'].str[:2]

# Keep rows where either 'bvd_country_code' or 'shareholder_country_code' is in the list of European countries
industry_classifications = test[
    test['subsidiary_country_code'].isin(european_countries) | 
    test['guo_country_code'].isin(european_countries)
]

industry_classifications

In [ ]:
test['type_of_relation'].unique()
test = test[~test['type_of_relation'].str.contains('DUO')]

In [ ]:
test[~test['type_of_relation'].str.contains('DUO') & test['guo_25'].notnull()]['direct_only_figures_']

In [ ]:
files = [f for f in os.listdir('firmografics\\all_addresses') if f.endswith('.parquet')]
full_paths = [os.path.join('firmografics\\all_addresses', f) for f in files]

all_addresses = pd.read_parquet(full_paths[0])

In [ ]:
len(full_paths)

In [ ]:
len(all_addresses.bvd_id_number.unique())

In [ ]:
files = [f for f in os.listdir('firmografics\\contact_info') if f.endswith('.parquet')]
full_paths = [os.path.join('firmografics\\contact_info', f) for f in files]

# contact_info = pd.read_parquet(full_paths[0])

In [ ]:
files = [f for f in os.listdir('firmografics\\industry_classifications') if f.endswith('.parquet')]
full_paths = [os.path.join('firmografics\\industry_classifications', f) for f in files]

industry_classifications = pd.read_parquet(full_paths[0])

_________________

In [ ]:
from pathlib import Path


In [ ]:
output_dir = Path('stata_files\\firmografics\\all_addresses')
output_dir.mkdir(parents=True, exist_ok=True)
input_dir = 'firmografics\\all_addresses'

files = [f for f in os.listdir(input_dir) if f.endswith('.parquet')]
full_paths = [os.path.join(input_dir, f) for f in files]


In [ ]:
for idx, file in enumerate(full_paths):
    df = pd.read_parquet(file)
    df = df[['bvd_id_number', 'postcode', 'country_iso_code']]
    df.to_csv(f'stata_files\\firmografics\\all_addresses\\chunk_{idx}.csv', index=False)


In [ ]:
output_dir = Path('stata_files\\firmografics\\contact_info')
output_dir.mkdir(parents=True, exist_ok=True)
input_dir = 'firmografics\\contact_info'

files = [f for f in os.listdir(input_dir) if f.endswith('.parquet')]
full_paths = [os.path.join(input_dir, f) for f in files]


In [ ]:
def dms_to_decimal(dms):
    import re
    try:
        pattern = re.compile(r'(\d+)°\s*(\d+)\'\s*([\d.]+)"\s*([NSEW])')
        match = pattern.match(dms)
        
        if match:
            degrees, minutes, seconds, direction = match.groups()
            decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
            
            if direction in ['S', 'W']:
                decimal = -decimal
            
            return decimal
        else:
            return np.nan 
    except Exception:
        return np.nan  


In [ ]:
for file in full_paths:
    df = pd.read_parquet(file)
    variables = ['bvd_id_number', 'country_iso_code', 'latitude', 'longitude']
    df = df[variables]
    df['latitude'] = df['latitude'].apply(dms_to_decimal)
    df['longitude'] = df['longitude'].apply(dms_to_decimal)
    df.to_csv(f'stata_files\\firmografics\\contact_info\\chunk_{idx}.csv', index=False)

In [ ]:
file1['latitude'].apply(dms_to_decimal)
file1['longitude'].apply(dms_to_decimal)

In [ ]:
1file1['latitude'] = file1['latitude'].apply(dms_to_decimal)
file1['longitude'] = file1['longitude'].apply(dms_to_decimal)
file1
